In [1]:
import pandas as pd
import numpy as np

import re

from utils.wordle import Wordle
import os

# playing wordle by statistics

In [2]:
encoding_table = {
    '2': 'O',
    '1': '-',
    '0': 'X'
}


def encode_score(score, encoding_table):
    score_str = "".join([str(i) for i in score])
    for key in list(encoding_table.keys()):
        score_str = re.sub(key, encoding_table[key], score_str)
    return score_str


def decode_score(score_str, encoding_table):
    score = []
    decode_table = dict()
    for key in encode_table.keys():
        decode_table[encode_table[key]] = key
    for s in score_str:
        score.append(int(decode_table[s]))
    return score

In [3]:
# Wordle Game Setting
n_letter = 5

wordle_file = f"./data/common {n_letter}-letter words.csv"

wd = Wordle(wordle_file)
wd.ans

'slots'

In [4]:
df = pd.read_csv(wordle_file)
df = df.dropna()
df

,word,count
0,aaron,7
1,aback,2
2,abbas,2
3,abbey,7
4,abbot,2
...,...,...
4067,ziggy,1
4068,zoned,1
4069,zones,3
4070,zooey,1


# Create Score Matrix

In [5]:
score_file = f"./guess score matrix {n_letter}-letter words.csv"

vocab = list(df['word'])

if not os.path.exists(score_file):
    score_matrix = []
    for ans in df['word']:

        # using function from wordle core
        wd = Wordle(wordle_file)

        # score for this ans
        score_arr = []
        for guess in df['word']:
            encoded_score = encode_score(wd.get_score(ans, guess), encoding_table)
            score_arr.append(encoded_score)
        score_matrix.append(score_arr)
        
    df = pd.DataFrame(columns=vocab, data=score_matrix)
    df.to_csv(score_file, index=False)
    
df = pd.read_csv(score_file)

In [6]:
def filter_possible_answer(df, possible_index, guess, guess_result):
    # find new possible answer (Vi - vocab based on guess turn i)
    # and new possible index
    # based on current guess (gi) that picked from previous vocab(Vi-1)
    
    # previous vocab(Vi-1)
    pos_df = df.loc[possible_index]
    
    # filter new answer
    pos_df = pos_df[pos_df[guess] == guess_result][guess]
    possible_words = list(df.columns[pos_df.index])
    
    return pos_df.index, possible_words


# Play

In [7]:
wd = Wordle(wordle_file)
print("(hidden) answer:", wd.ans)
print("")


# first guess
guess = None
correct = False
possible_ind = range(len(df))

while not(correct):
    # guess turn i
    
    # the first turn pick a word from V0 i.e. vocab
    # pick word from V(i-1), based on previous possible vocab
    if guess == None:
        guess = np.random.choice(vocab, 1)[0]
    else:
        guess = np.random.choice(possible_words, 1)[0]
        
    wd_result = wd.guess(guess)
    guess_result = encode_score(wd_result['score'], encoding_table)
    
    # create new vocab based on guess turn i and score turn i
    possible_ind, possible_words = filter_possible_answer(df, possible_ind, guess, guess_result)
    
    correct = (wd_result['score'] == 2).all()
    
    print(f"guess: {wd_result['#guess']} | {wd_result['guessed word']} | {''.join([str(i) for i in wd_result['score']])}")
    print(f"words left: {len(possible_words)}")
    if len(possible_words) < 10:
        print(f"{possible_words}")
    print("")

(hidden) answer: yelps

guess: 1 | swing | 10000
words left: 599

guess: 2 | dumps | 00022
words left: 20

guess: 3 | claps | 01022
words left: 3
['helps', 'loops', 'yelps']

guess: 4 | yelps | 22222
words left: 1
['yelps']

